In [1]:
from tools import *
import os
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from tqdm import tqdm  
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler



Préparation et chargement des graphes pour l'entraînement du GNN.

In [2]:
graph_json_dir = "graphes_JSON_Complet"  
graph_json_files = [graph_json_dir+"/"+f for f in os.listdir(graph_json_dir) if f.endswith('.json')]

all_graphs = []

for path in graph_json_files:
    node_features_df=extract_node_features_from_json_file(path)
    edges_df=extract_mapped_edges_from_json(path)
    target_df=extract_optimal_repartition_from_json(path)

    node_features_tensor, edge_index_tensor, y_target_tensor = prepare_data_for_GNN(node_features_df, edges_df, target_df)
 
    data = Data(x=node_features_tensor, edge_index=edge_index_tensor, y=y_target_tensor)

    all_graphs.append(data)


print("finished data preparting & loading")

finished data preparting & loading


In [3]:
len(all_graphs)


4998

separer les données en train validation & test

In [4]:
from torch.utils.data import random_split

total_graphs = len(all_graphs)
train_size = int(0.8 * total_graphs)
val_size = int(0.1 * total_graphs)
test_size = total_graphs - train_size - val_size

train_data, val_data, test_data = random_split(all_graphs, [train_size, val_size, test_size])

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8)
test_loader = DataLoader(test_data, batch_size=8)

print(f"Train set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")


Train set size: 3998
Validation set size: 499
Test set size: 501


In [23]:
grapheIndex=3452
print(all_graphs[grapheIndex].x.shape,all_graphs[grapheIndex].y.shape)

torch.Size([124, 8]) torch.Size([124])


Model training

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class GraphSAGE(torch.nn.Module):
    def __init__(self, num_features, hidden_dim=64,hidden_dim_2=64,hidden_dim_3=100):
        super().__init__()
        self.conv1 = SAGEConv(num_features, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim_2)
        self.conv3 = SAGEConv(hidden_dim_2, hidden_dim_3) 
        self.lin = torch.nn.Linear(hidden_dim_3, hidden_dim_3)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = torch.relu(self.conv1(x, edge_index))
        # print("After conv1:", x.shape)
        x = torch.dropout(x, p=0.01, train=self.training)
        x = torch.relu(self.conv2(x, edge_index))
        # print("After conv2:", x.shape)
        x = torch.dropout(x, p=0.01, train=self.training)
        x = torch.relu(self.conv3(x, edge_index))
        # print("After conv3:", x.shape)
        x = self.lin(x)
        # print("After linear:", x.shape)
        return x



SyntaxError: invalid decimal literal (1133516175.py, line 17)

In [ ]:
device = torch.device('cpu')
model = GraphSAGE(num_features=8).to(device)  
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)


In [ ]:
for epoch in range(100):
    model.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0

    for data in train_data:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        targets = data.y.long()
        loss = F.cross_entropy(out, targets)
        loss.backward()
        optimizer.step()

        _, pred = out.max(dim=1)
        correct = (pred == targets).sum().item()

        total_loss += loss.item()
        total_correct += correct
        total_samples += targets.size(0)

    if (epoch % 10 == 0) or (epoch == 99):
        epoch_loss = total_loss / len(train_data)
        epoch_acc = total_correct / total_samples
        print(f'Epoch {epoch+1:3d} | Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f}')


Epoch 001 | Loss: 3.9886 | Acc: 0.0762
Epoch 011 | Loss: 3.4719 | Acc: 0.0905
Epoch 021 | Loss: 3.4341 | Acc: 0.0909
Epoch 031 | Loss: 3.4374 | Acc: 0.0918
Epoch 041 | Loss: 3.4100 | Acc: 0.0915
Epoch 051 | Loss: 3.4220 | Acc: 0.0932
Epoch 061 | Loss: 3.4020 | Acc: 0.0925
Epoch 071 | Loss: 3.4185 | Acc: 0.0928
Epoch 081 | Loss: 3.3872 | Acc: 0.0915
Epoch 091 | Loss: 3.4074 | Acc: 0.0924
Epoch 100 | Loss: 3.4030 | Acc: 0.0923
